In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write





def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


## Kinyarwanda

In [7]:
# common_voice_rw_21176283.wav
text = '''Hello I'm navneeth'''


# DUMMY3/common_voice_rw_21072591.wav|
text2 = '''Hello I'm nav'''

In [8]:
hps = utils.get_hparams_from_file("configs/ljs_base.json")

In [9]:


net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()


_ = utils.load_checkpoint("/home/navneeth/EgoPro/dnn/vits_infer/models/ljspeech.pth", net_g, None)

INFO:root:Loaded checkpoint '/home/navneeth/EgoPro/dnn/vits_infer/models/ljspeech.pth' (iteration 998)


In [10]:
l = [text, text2]
directory = "kinyarwanda"
os.makedirs(directory,exist_ok=True)
for i in range(len(l)):
    stn_tst = get_text(l[i], hps)
    with torch.no_grad():
        x_tst = stn_tst.cuda().unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cpu()
        audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
    write(f"{directory}/old_d{i}.mp3",rate=hps.data.sampling_rate,data=audio)

TypeError: text_to_sequence() missing 1 required positional argument: 'lang'